# Data exploration

In [1]:
import pandas as pd
import numpy as np
from helpers import count_unique_words
from gensim.models import Word2Vec
import gensim

/home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/stefano/anaconda3/envs/tf-gpu/lib/pyt

In [21]:
df = pd.read_pickle("dataframes/full_df_cleaned_train_0_8.pickle")

In [3]:
df.head()

,sentence,label
1128954,follow naked cheetah goodness sake ! get !,1
1419820,turn pbo computer feed kept stopping,-1
1182790,battle sex rally friday male wear blue support...,1
1954504,work dean ing annotated bibliography guide bib...,-1
72552,! speakinqq real ishh toniqhtt tho !,1


In [4]:
my_value_counts = df.iloc[:].sentence.apply(lambda x: x.split()).explode().value_counts()

In [46]:
print(len(my_value_counts[my_value_counts < 2]))
print(len(my_value_counts))

277617
452520


In [31]:
hashtags_count = 0
hashtags = []
for el in my_value_counts.index:
    if str.startswith(el, "#"):
        hashtags_count += 1
        hashtags.append(el)

In [36]:
my_value_counts[hashtags][my_value_counts<10]

#blondemoment            9
#theelites               9
#61khunter               9
#thankspep               9
#kyuevilquotes           9
                        ..
#weloveshout             1
#sinner                  1
#letshaveahouseparty     1
#foreverdisliked         1
#favoritepartoftheday    1
Name: sentence, Length: 92774, dtype: int64

In [5]:
df_new = df.sentence.apply(lambda x: " ".join([el if my_value_counts[el]>1 else "" for el in x.split()]))

In [6]:
df.sentence = df_new

In [8]:
df.to_pickle("dataframes/df_train_0_8_more_than_1_occurrence.pickle")

In [12]:
w2v_model = gensim.models.KeyedVectors.load_word2vec_format("models/GoogleNews-vectors-negative300.bin", binary=True)

In [44]:
unique_words = df['sentence'].apply(lambda x: x.split()).explode().unique()
len(unique_words)

66129

In [29]:
%time df_new = df['sentence'].apply(lambda x: " ".join([el if el in w2v_model.wv.vocab else "" for el in x.split()]))

/home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


CPU times: user 27.4 s, sys: 35.9 ms, total: 27.5 s
Wall time: 27.5 s


In [37]:
df_new

1128954             follow naked cheetah goodness sake  get 
1419820                    turn  computer feed kept stopping
1182790    battle sex rally friday male wear blue support...
1954504    work dean ing annotated bibliography guide bib...
72552                                            real   tho 
                                 ...                        
1436823    may possibly new best friend   sa perk wallflo...
41974                r u shaving sweet cunt laser depilated 
1112919    nice way end long day wanna go beach soo badly...
1281173    highly traded stock c nok   qq following stock...
429135          even know begin  think later something like 
Name: sentence, Length: 2000000, dtype: object

In [38]:
df.sentence = df_new
print(df)

                                                  sentence  label
1128954           follow naked cheetah goodness sake  get       1
1419820                  turn  computer feed kept stopping     -1
1182790  battle sex rally friday male wear blue support...      1
1954504  work dean ing annotated bibliography guide bib...     -1
72552                                          real   tho       1
...                                                    ...    ...
1436823  may possibly new best friend   sa perk wallflo...     -1
41974              r u shaving sweet cunt laser depilated       1
1112919  nice way end long day wanna go beach soo badly...      1
1281173  highly traded stock c nok   qq following stock...     -1
429135        even know begin  think later something like       1

[2000000 rows x 2 columns]


In [39]:
len(df[df.sentence == ''])

7624

In [40]:
new_value_counts = df.iloc[:]['sentence'].apply(lambda x: x.split()).explode().value_counts()

In [42]:
new_value_counts[new_value_counts==1]

screensavers    1
implicitly      1
yomp            1
globale         1
antigravity     1
               ..
hepatic         1
samll           1
shedule         1
nannying        1
postmistress    1
Name: sentence, Length: 13095, dtype: int64

In [43]:
df.to_pickle("dataframes/dataframe_train_0_8_google.pickle")

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle
import re
import multiprocessing

from helpers import count_unique_words, count_unique_ngrams, \
            build_unique_ngrams, create_sentence_vectors, create_sentence_vectors_submission, \
            create_sentence_chi2_vectors

import sys

import gensim   # Not sure whether it is better to use gensim or tensorflow :/
import logging
from gensim.models.phrases import Phrases, Phraser

from gensim.models import Word2Vec
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 
sys.path.append('../')

/home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/stefano/anaconda3/envs/tf-gpu/lib/pyt

In [2]:
w2v_model = Word2Vec.load("models/w2v_model_epochs_5_win_5_sg_200.model")

2019-12-18 02:35:46,525 : INFO : loading Word2Vec object from models/w2v_model_epochs_5_win_5_sg_200.model
2019-12-18 02:35:47,236 : INFO : loading wv recursively from models/w2v_model_epochs_5_win_5_sg_200.model.wv.* with mmap=None
2019-12-18 02:35:47,237 : INFO : loading vectors from models/w2v_model_epochs_5_win_5_sg_200.model.wv.vectors.npy with mmap=None
2019-12-18 02:35:47,336 : INFO : setting ignored attribute vectors_norm to None
2019-12-18 02:35:47,336 : INFO : loading vocabulary recursively from models/w2v_model_epochs_5_win_5_sg_200.model.vocabulary.* with mmap=None
2019-12-18 02:35:47,337 : INFO : loading trainables recursively from models/w2v_model_epochs_5_win_5_sg_200.model.trainables.* with mmap=None
2019-12-18 02:35:47,337 : INFO : loading syn1neg from models/w2v_model_epochs_5_win_5_sg_200.model.trainables.syn1neg.npy with mmap=None
2019-12-18 02:35:47,430 : INFO : setting ignored attribute cum_table to None
2019-12-18 02:35:47,430 : INFO : loaded models/w2v_model_epo

In [3]:
word_vector_size = w2v_model.wv.vectors.shape[1]
word_vector_size

200

In [4]:
train_df = pd.read_pickle("dataframes/dataframe_train_0_8_google.pickle")

In [5]:
chi2_df = pd.read_pickle("dataframes/chi2_google_words.pickle") 

In [6]:
limit = 200

In [7]:
x = train_df.iloc[:limit]['sentence']
y = train_df.iloc[:limit]['label']
y = y.where(y == 1, 0) 

2019-12-18 02:35:48,249 : INFO : NumExpr defaulting to 8 threads.


In [8]:
print(x)
print(y)

1128954             follow naked cheetah goodness sake  get 
1419820                    turn  computer feed kept stopping
1182790    battle sex rally friday male wear blue support...
1954504    work dean ing annotated bibliography guide bib...
72552                                            real   tho 
                                 ...                        
121096     nothing like taking afternoon escort tech get ...
2138906             ughh feeling awful  barely even eat soup
1334637    ha programming lost way  reflexion chris bos p...
1339863    fundamental wimax understanding broadband wire...
2111715           scared wa waving friend u wa like hahahaha
Name: sentence, Length: 200, dtype: object
1128954    1
1419820    0
1182790    1
1954504    0
72552      1
          ..
121096     1
2138906    0
1334637    0
1339863    0
2111715    0
Name: label, Length: 200, dtype: int64


In [9]:
chi2_df.set_index("word", inplace=True)

In [19]:
chi2_df.loc['get'].isnull()[0]

True

In [20]:
from helpers import *
sentence_x, sentence_y = create_sentence_chi2_vectors(x, y, word_vector_size, w2v_model, chi2_df)

0it [00:00, ?it/s]

Computed the average vector


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().